In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import time

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [2]:
gdf = aup.gdf_from_db("mpos_2020", "marco")
m = ['01001']
mun_gdf = gpd.GeoDataFrame()
for mun in m:
    mun_gdf = mun_gdf.append(gdf.loc[gdf.CVEGEO == mun])
mun_gdf.head(1)

,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry
0,01001,01,001,Aguascalientes,"POLYGON ((-102.10641 22.06035, -102.10368 22.0..."


In [3]:
poly = mun_gdf.geometry
# Extracts coordinates from polygon as DataFrame
coord_val = poly.bounds
# Gets coordinates for bounding box
n = coord_val.maxy.max()
s = coord_val.miny.min()
e = coord_val.maxx.max()
w = coord_val.minx.min()

In [4]:
G = ox.graph_from_bbox(n, s, e, w, network_type="all")
# Transforms graph to nodes and edges GeoDataFrame
#nodes, edges = ox.graph_to_gdfs(G)

In [5]:
# Creates query to download nodes from the metropolitan area or capital
poly_wkt = mun_gdf.dissolve().geometry.to_wkt()[0]
query = f"SELECT * FROM denue.denue_2020 WHERE ST_Intersects(geometry, \'SRID=4326;{poly_wkt}\')"
denue = aup.gdf_from_query(query, geometry_col='geometry')
print(denue.shape)
denue.head(2)

(46054, 42)


,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,...,ageb,manzana,telefono,correoelec,www,tipo_uni_eco,latitud,longitud,fecha_alta,geometry
0,6862345.0,AGROPECUARIA CHARCOS DE QUEZADA,AGROPECUARIA CHARCOS DE QUEZADA SOCIEDAD DE PR...,115119.0,Otros servicios relacionados con la agricultura,31 a 50 personas,AVENIDA,UNIVERSIDAD,CALLE,BENJAMÍN MÉNDEZ,...,0233,9.0,4492854000.0,MICHAEL.GALINDO@UGASA.COM.MX,None,Fijo,21.906992,-102.30981,2019-04,POINT (-102.30981 21.90699)
1,6905212.0,AGROPECUARIO LOS ALAMITOS,None,115119.0,Otros servicios relacionados con la agricultura,0 a 5 personas,CALLE,J. ENCARNACION PAYEN,CALLE,JARDINES ETERNOS,...,0322,18.0,4499960300.0,None,None,Fijo,21.889927,-102.31401,2019-11,POINT (-102.31401 21.88993)


In [6]:
start = time.time()
gdf_old = aup.find_nearest_old(G, denue, 'all')
end = time.time()
print(end - start)

/root/anaconda3/envs/geo_env/lib/python3.9/site-packages/osmnx/distance.py:426: UserWarning: The `get_nearest_nodes` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2.4730117321014404


In [15]:
start = time.time()
gdf_new = aup.find_nearest(G, denue)
end = time.time()
print(end - start)

2.5138957500457764
